In [29]:
import boto3
from botocore.exceptions import ClientError
import logging
import os
import pandas as pd
from dotenv import load_dotenv

In [30]:
load_dotenv('.env')

True

In [31]:
s3 = boto3.client('s3',
   aws_access_key_id= os.getenv('aws_access_key_id'),
   aws_secret_access_key=os.getenv('aws_secret_access_key'),
   aws_session_token=os.getenv('aws_session_token'),
   region_name='us-east-1'
)

In [32]:
bucket_name = 'desafio-thomas'
s3_file_name = 'dados.csv'
objeto = s3.get_object(Bucket=bucket_name, Key=s3_file_name)
database = pd.read_csv(objeto['Body'], sep=';')

In [33]:
database = database.drop(['Revenda', 'CNPJ da Revenda', 'Nome da Rua', 'Numero Rua', 'Complemento', 'Bairro', 'Cep'], axis=1)

## Media de Valor de Venda nas regioes SE e NE

In [34]:
database['Valor de Venda'] = database['Valor de Venda'].str.replace(r',', '.', regex=False)
database['Valor de Venda'] = database['Valor de Venda'].astype(float)

In [35]:
filter1 = database[['Regiao - Sigla', 'Valor de Venda']].loc[(database['Regiao - Sigla'] == 'SE') | (database['Regiao - Sigla'] == 'NE')].groupby('Regiao - Sigla').mean()\
.to_csv('media_regioes.csv', index=False, encoding='utf-8-sig')

In [36]:
file_name = 'media_regioes.csv'
s3_path = 'media_regioes.csv'

try:
    s3.upload_file(file_name, bucket_name, s3_path)
except ClientError as e:
    logging.error(e)

## Quantidade de vezes que foi registrado 'Gasolina' em cada estado

In [37]:
filter2 = database[['Regiao - Sigla', 'Produto']].loc[database['Produto'] == 'GASOLINA'].groupby('Regiao - Sigla').count().to_csv('gasolina_por_regiao.csv', encoding='utf-8-sig')

In [38]:
file_name = 'gasolina_por_regiao.csv'
s3_path = 'gasolina_por_regiao.csv'

try:
    s3.upload_file(file_name, bucket_name, s3_path)
except ClientError as e:
    logging.error(e)

## Os 10 maiores lucros por litro (Preço de compra - preço de venda)

In [39]:
filter = database.copy()
filter['Valor de Compra'] = filter['Valor de Compra'].str.replace(r',', '.', regex=False)
filter['Valor de Compra'] = filter['Valor de Compra'].astype(float)

In [40]:
filter = filter.dropna(subset=['Valor de Venda', 'Valor de Compra'])
filter = filter.drop_duplicates()

In [41]:
filter['Lucro'] = filter['Valor de Venda'] - filter['Valor de Compra']

In [42]:
filter3 = filter[['Estado - Sigla', 'Municipio', 'Produto', 'Lucro']].sort_values('Lucro', ascending=False).head(10).to_csv('top_10_lucro.csv', index=False, encoding='utf-8-sig')

In [43]:
file_name = 'top_10_lucro.csv'
s3_path = 'top_10_lucro.csv'

try:
    s3.upload_file(file_name, bucket_name, s3_path)
except ClientError as e:
    logging.error(e)